In [1]:
from os import listdir
from os.path import isdir, join
from tensorflow.keras import layers, models
import numpy as np

In [2]:
dataset_path = 'Documents\speech_commands_v0.01'
all_targets = [name for name in listdir(dataset_path) if isdir(join(dataset_path, name))]
print(all_targets)

['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero', '_background_noise_']


In [3]:
commands = [all_targets[i] for i in [4,11,18,22,26]]
commands

['down', 'left', 'right', 'stop', 'up']

In [4]:
# Settings
feature_sets_filename = 'commands_mfcc_sets.npz'
model_filename ='command_detection_model.h5'
labels = ['down', 'left', 'right', 'stop', 'up']

In [5]:
feature_sets = np.load(feature_sets_filename)
print(feature_sets.files)

['x_train', 'y_train', 'x_val', 'y_val', 'x_test', 'y_test']


In [6]:
x_train = feature_sets['x_train']
y_train = feature_sets['y_train']
x_val = feature_sets['x_val']
y_val = feature_sets['y_val']
x_test = feature_sets['x_test']
y_test = feature_sets['y_test']

In [29]:
# CNN for TF expects (batch, height, width, channels)
# So we reshape the input tensors with a "color" channel of 1
x_train = x_train.reshape(x_train.shape[0], 
                          x_train.shape[1], 
                          x_train.shape[2], 
                          1)
x_val = x_val.reshape(x_val.shape[0], 
                      x_val.shape[1], 
                      x_val.shape[2], 
                      1)
x_test = x_test.reshape(x_test.shape[0], 
                        x_test.shape[1], 
                        x_test.shape[2], 
                        1)
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(8726, 16, 16, 1)
(1085, 16, 16, 1)
(1100, 16, 16, 1)


In [30]:
sample_shape = x_test.shape[1:]
print(sample_shape)

(16, 16, 1)


In [37]:
model = models.Sequential()
model.add(layers.Conv2D(32, 
                        (2, 2), 
                        activation='relu',
                        input_shape=sample_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Classifier
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='softmax'))

In [38]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='rmsprop', 
              metrics=['acc'])

In [39]:
history = model.fit(x_train, 
                    y_train, 
                    epochs=30, 
                    batch_size=100, 
                    validation_data=(x_val, y_val))

Epoch 1/30
88/88 [==============================] - 1s 17ms/step - loss: 0.0194 - acc: 0.1984 - val_loss: 6.5063 - val_acc: 0.2009
Epoch 2/30
88/88 [==============================] - 1s 13ms/step - loss: 1.2748e-04 - acc: 0.1997 - val_loss: 11.1215 - val_acc: 0.2009
Epoch 3/30
88/88 [==============================] - 1s 13ms/step - loss: 7.2720e-05 - acc: 0.1997 - val_loss: 17.5763 - val_acc: 0.2009
Epoch 4/30
88/88 [==============================] - 1s 14ms/step - loss: 1.2886e-07 - acc: 0.1997 - val_loss: 19.0294 - val_acc: 0.2009
Epoch 5/30
88/88 [==============================] - 1s 14ms/step - loss: 5.0043e-04 - acc: 0.1996 - val_loss: 19.3728 - val_acc: 0.2009
Epoch 6/30
88/88 [==============================] - 1s 14ms/step - loss: 4.0024e-08 - acc: 0.1997 - val_loss: 20.1075 - val_acc: 0.2009
Epoch 7/30
88/88 [==============================] - 1s 14ms/step - loss: 7.6581e-07 - acc: 0.1997 - val_loss: 25.4510 - val_acc: 0.2009
Epoch 8/30
88/88 [==============================] - 1

In [15]:
model.evaluate(x_test, y_test)

35/35 [==============================] - 0s 3ms/step - loss: -7028546207744.0000 - acc: 0.2164


[-7028546207744.0, 0.21636363863945007]